In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Practicum/v1/train_df.csv')

Define the features 'X' and target variable 'y'

In [ ]:
X = df.drop('TOTALPMT_ADJ',axis=1).drop('LOG_TOTALPMT', axis=1)
y = df[['TOTALPMT_ADJ']]

I. Hyperparameter Tuning

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
# define the search space for gridsearch
search_space = {
    'n_estimators': [50, 75, 100],
}

In [ ]:
ada = AdaBoostRegressor(
    # the hyperparameters are based on gridsearch cv done previously for random forest regressor
    RandomForestRegressor(max_depth=15, n_estimators=100),
)

In [ ]:
GS_object = GridSearchCV(
    ada, 
    param_grid = search_space,
    scoring = 'neg_root_mean_squared_error',
    cv = 3, # cv = k-fold cross validation. The input is k.
    verbose = 4,
    n_jobs= -1)

In [ ]:
# again, gridsearch will automatically conduct k-fold and determine the optimal parameters using k train and validation sets
GS_object.fit(X, y)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


KeyboardInterrupt: ignored

Save to csv

In [ ]:
gs_df = pd.DataFrame(GS_object.cv_results_)
gs_df = gs_df.sort_values('rank_test_r2')
gs_df.to_csv('/content/drive/MyDrive/Practicum/v1/gs_df_ada.csv')

III. Model Evaluation

Define the KFold splits

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
n_splits = 5
cv = KFold(n_splits)

In [ ]:
ada_optim = AdaBoostRegressor(
    DecisionTreeRegressor(max_depth = 15),
    n_estimators = 100, 
    learning_rate=.5)

Evaluate Errors

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
errors = []
for train_index, test_index in cv.split(X):
    X_train_kfold, y_train_kfold = X.iloc[train_index], y.iloc[train_index]
    X_test_kfold, y_test_kfold = X.iloc[test_index], y.iloc[test_index]
    ada_optim.fit(X_train_kfold, y_train_kfold)
    y_pred = ada_optim.predict(X_test_kfold)
    errors.append(np.sqrt(mean_squared_error(y_test_kfold, y_pred)))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

In [ ]:
errors 

[328415.50777647435,
 325285.58628037,
 327781.0433801931,
 325094.5360263175,
 326751.6349728988]

In [ ]:
np.mean(errors)

326665.6616872507

In [ ]:
er_df = pd.DataFrame({'errors': errors})
er_df.to_csv('/content/drive/MyDrive/Practicum/v1/er_df_ada.csv')

Final Eval

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/Practicum/v1/test_df.csv')

In [ ]:
X_test = test_df.drop('TOTALPMT_ADJ',axis=1).drop('LOG_TOTALPMT', axis=1)
y_test = test_df[['TOTALPMT_ADJ']]

In [ ]:
ada_optim.fit(X, y)
y_pred = ada_optim.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


330424.50240746647